In [1]:
import pandas as pd
import  numpy as np 
import math

In [3]:
train=pd.read_csv('trainKinetic.csv')

In [14]:
train.head(3)

id   timestamp  full_sq  life_sq  floor  max_floor  material  build_year  \
0   1  2011-08-20       43     27.0    4.0    0.10838  0.493169     0.01998   
1   2  2011-08-23       34     19.0    3.0    0.10838  0.493169     0.01998   
2   3  2011-08-27       43     29.0    2.0    0.10838  0.493169     0.01998   

   num_room  kitch_sq    ...      cafe_count_5000_price_2500  \
0  0.334164  0.126191    ...                               9   
1  0.334164  0.126191    ...                              15   
2  0.334164  0.126191    ...                              10   

  cafe_count_5000_price_4000 cafe_count_5000_price_high  \
0                          4                          0   
1                          3                          0   
2                          3                          0   

   big_church_count_5000  church_count_5000  mosque_count_5000  \
0                     13                 22                  1   
1                     15                 29                  1   
2                     11                 27                  0   

   leisure_count_5000  sport_count_5000  market_count_5000  price_doc  
0                   0                52                  4    5850000  
1                  10                66                 14    6000000  
2                   4                67                 10    5700000  

[3 rows x 272 columns]

In [5]:
import scipy.stats

In [8]:
scipy.stats.pearsonr(train['price_doc'],train['full_sq'])

(0.34184045975381705, 0.0)

In [9]:
scipy.stats.pearsonr(train['price_doc'],train['life_sq'])

(0.1717147631802001, 2.5351155712070379e-200)

In [10]:
scipy.stats.pearsonr(train['price_doc'],train['num_room'])                                                 

(0.35748423509921823, 0.0)

In [11]:
scipy.stats.pearsonr(train['full_sq'],train['num_room'])    

(0.23738723337722459, 0.0)

In [215]:
class Cluster :
    def __init__(self,full ,room,idx):
        """
        Created a cluster  based on  full_sq and  num_room
        """
        self._idx={idx}
        self._full=full
        self._room=room
        #self._idx.append(idx)#adding  ids   to    be able  to   flag then  the   features  based on corresponding clustering
    def  full(self):
        return self._full
    def  room(self):
        return self._room
    def  idx(self):
        return self._idx
    def copy(self):
        """
        returns  a copy of a  cluster
        """
        copy_cluster=Cluster(self._idx,self._room, self._idx)
    def distance(self,other_cluster):
        """
        computes     euclidean distance between  2  clusters 
        """
        x_dist=self._full-other_cluster.full()
        y_dist=self._room-other_cluster.room()
        return  math.sqrt(x_dist**2+y_dist**2)
    def merge_clusters(self,other_cluster):
        """
        Merge one cluster   into another
        """
        if len(other_cluster._idx)==0:
            return  self
        else:
            #add  idx :
            self._idx=self._idx.union(other_cluster.idx())
            #compute new cluster  centers
            """ The  centroid  of new cluster    is   give  by:
            the point(X',Y'),
            where :
            X':=sum_from_i_to_m(Xi)/m
            Y':=sum_from_i_to_m(Yi)/m       
            recall that full from   full_sq and  room  from   num_romm  features  are  our points
            """
            self._full=(self._full+other_cluster.full())/2
            self._room=(self._room+other_cluster.room())/2
            return self

In [216]:
#Loocking at some examples

In [217]:
train[['id','full_sq','num_room']].head(3)

id  full_sq  num_room
0   1       43  0.334164
1   2       34  0.334164
2   3       43  0.334164

In [218]:
point=train.loc[0,['id','full_sq','num_room']]#brings a series
point

id                 1
full_sq           43
num_room    0.334164
Name: 0, dtype: object

In [219]:
point_0=Cluster(point[1],point[2],point[0])

In [220]:
point_0.idx()

{1}

In [221]:
point_0.full()

43

In [222]:
point_0.room()

0.33416368377055905

In [223]:
point=train.loc[1,['id','full_sq','num_room']]#brings a series
print(point)
point_1=Cluster(point[1],point[2],point[0])

id                 2
full_sq           34
num_room    0.334164
Name: 1, dtype: object


In [224]:
point_1.idx()

{2}

In [225]:
###################################testing if  THE  merging is OK =)))############################################

In [226]:
point_0.distance(point_1)

9.0

In [227]:
new_clust=point_0.merge_clusters(point_1)

In [228]:
new_clust._idx

{1, 2}

In [229]:
new_clust.room()

0.33416368377055905

In [230]:
new_clust.full()

38.5

In [232]:
####################################################################################################################

In [233]:
import math

In [237]:
import  math
def slow_closest_pair(cluster_list):
    minimum_point=(math.inf,-1,-1)
    for  index1 in range(0,len(cluster_list)):
        for index2 in  range(0,len(cluster_list)):
          
            if index1<index2:
                
                if cluster_list[index1].distance(cluster_list[index2])<minimum_point[0]:
                    minimum_point=(cluster_list[index1].distance(cluster_list[index2]),index1,index2)
        
    return minimum_point

In [310]:
point=train.loc[200,['id','full_sq','num_room']]#brings a series
print(point)
point_2=Cluster(point[1],point[2],point[0])
cluster_list=[point_0,point_1,point_2]

id               201
full_sq           38
num_room    0.334164
Name: 200, dtype: object


In [291]:
slow_closest_pair(cluster_list)

(1.875, 0, 2)

In [299]:
def hierarchical_clustering(cluster_list, num_clusters) :
  
    clusters=cluster_list
    while len(clusters)>num_clusters:
 
        closest_points=slow_closest_pair(clusters)
        clusters[closest_points[1]].merge_clusters(clusters[closest_points[2]])
        clusters.pop(closest_points[2])
 
    return cluster_list

In [311]:
hc=hierarchical_clustering(cluster_list, 2)

In [320]:
cluster_list=[]
for  row in range(train.shape[0]):
    point=train.loc[row,['id','full_sq','num_room']]
    point_cluster=Cluster(point[1],point[2],point[0])
    cluster_list.append(point_cluster)
 
    

In [ ]:
hc=hierarchical_clustering(cluster_list, 13)